In [1]:
import json
import numpy as np
from sklearn.metrics.pairwise import distance_metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import urllib2
import csv

In [8]:
# 'https://www.airbnb.com/rooms/825531?s=vJx50hcn'

headers = {"User-agent":"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"}
search = input('Enter url: ')
if (search.find('?') == -1):
    search_id = search[search.find('rooms/')+6:]
else:
    search_id = search[search.find('rooms/')+6:search.find('?')]
url = 'https://api.airbnb.com/v2/listings/' + search_id + '?client_id=3092nxybyb0otqw18e8nh5nty&_format=v1_legacy_for_p3'
request = urllib2.Request(url, headers=headers)
data = json.loads(urllib2.urlopen(request).read())
input_description = data['listing']['description']
print(input_description)

Enter url: 'https://www.airbnb.com/rooms/825531?s=vJx50hcn'
Apartment is located in the residential neighborhood of Roscoe Village. There are restaurants and bars within a 10 minute walk, but you can easily hop on one of the nearby bus lines to reach any other neighborhood in the city. 

Furnished one bedroom apartment available for short term rent.  Free street parking and convenient to public transit. (Damen, Belmont, Diversey and Clybourn buses).

Bedroom features a full-size bed with fresh linens. Additional sleeping areas available on living room futon.

Wifi and basic cable. Fully equipped kitchen with microwave and AC for the Summer months. 

Located in Roscoe Village, a safe and active neighborhood easily accessible to Wrigley Field, Downtown, and popular bars and restaurants. Awesome taco joint open till 3 AM is my favorite go-to after a night out.

Please help yourself to toiletries in the bathroom and anything in the kitchen that you need for a comfortable stay.

I travel a 

In [9]:
#make a dictionary of descriptions: descriptions[listing_id] = description
#make a tf-idf matrix of words by listings
#cosine similarity

#load the data
with open('data/filtered_nyc_listings.csv') as f:
     NYClistings = [{k: v for k, v in row.items()}
          for row in csv.DictReader(f, skipinitialspace=True)]
        
with open('data/filtered_sf_listings.csv') as f:
     SFlistings = [{k: v for k, v in row.items()}
          for row in csv.DictReader(f, skipinitialspace=True)]
        
descript_dict = {}
for n in NYClistings:
    key = n['id']
    descript_dict[key]=n['description']
    
for s in SFlistings:
    key = s['id']
    descript_dict[key]=s['description']
    


In [40]:
n_feats = 5000
descriptions = [input_description]+ [descript_dict[d] for d in descript_dict]
listing_by_vocab = np.empty((len(descriptions), n_feats))


tfidf_vec = TfidfVectorizer(min_df = 10, max_df = .8, max_features = n_feats, norm='l2', stop_words = 'english')
listing_by_vocab = tfidf_vec.fit_transform(descriptions)

In [37]:
ranked_list = np.argsort(cosine_similarity(listing_by_vocab[0:1], listing_by_vocab))[0][::-1]
print(ranked_list)

[    0 12376 13084 ..., 28340  5581 36113]


In [39]:
top_ten = ranked_list[1:11] #first element is the input listing itself
top_ten_descriptions = []
for i in top_ten:
    top_ten_descriptions.append(descriptions[i])
print(top_ten_descriptions)

#get ids of the listings so can look up other information to display
#need to figure out a way to map back from description to id
#descript dict maps the id key to the description value
#is there a faster way to do cosine similarity? can we construct a tf-idf vector from the input description instead
#of adding it to the tf-idf matrix?

["Enjoy your very own duplex apartment, centrally located in the hip & historic Clinton Hill neighborhood. Walk the tree-lined streets of this fun and beautiful area in Brooklyn or hop on the subway one block away, and get to downtown Manhattan easily. Enjoy your very own duplex apartment located on a pleasant, tree lined side street. If you're up for a nice walk, you can easily get to Fort Greene Park, the Brooklyn Botanical Gardens, Prospect Park, The Brooklyn Museum or go to dozens of great restaurants and bars in the neighborhood. Want to travel further to see some of New York's amazing sites? The apartment is located one block away from a C train subway stop! It's just a few steps away! We are going on vacation, so we are renting out our lovely apartment for this short time only. The apartment is clean and spacious. On the upper level, there is an foyer/living room that has a futon on it, which can easily be converted into a bed. There is also the main bedroom and the bathroom on 